In [181]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/videogames-sales-dataset/XboxOne_GameSales.csv
/kaggle/input/videogames-sales-dataset/Video_Games_Sales_as_at_22_Dec_2016.csv
/kaggle/input/videogames-sales-dataset/PS4_GamesSales.csv


In [182]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.animation as animation
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
vgdata=pd.read_csv('../input/videogames-sales-dataset/Video_Games_Sales_as_at_22_Dec_2016.csv')

In [ ]:
vgdata.dropna(subset=['Name'],inplace=True)
vgdata.dropna(subset=['Year_of_Release'],inplace=True)
vgdata.dropna(subset=['Publisher'],inplace=True)
vgdata['Developer'].fillna('None',inplace=True)
vgdata['Rating'].fillna('None',inplace=True)

In [ ]:
vgdata.sample(5)

In [ ]:
vgdata.dtypes

In [ ]:
vgdata[vgdata['Platform']=='X360'].sort_values('Global_Sales',ascending=False).head(30)

In [ ]:
#Groupby best game
#vgdata.groupby('Platform')['Global_Sales', 'Name'].sum()
vgdata.groupby(['Platform','Name'], as_index=False)['Global_Sales'].max().sort_values("Global_Sales",ascending=False)

In [ ]:
data=vgdata.Rating.value_counts()
data.plot(kind='bar')

In [ ]:
vgdata[['Platform','Name','NA_Sales','EU_Sales','JP_Sales','Other_Sales']]\
.loc[vgdata.groupby('Platform').NA_Sales.agg('idxmax')]\
.sort_values('NA_Sales',ascending = False).set_index('Name').plot(kind='bar',stacked=True,figsize=(20,5))

In [ ]:
data=vgdata.Developer.value_counts()[1:20]
data.plot(kind='bar')

so 2007 to 2010 is the best year in gaming apparently

In [ ]:
genres=list(vgdata.Genre.unique())
del genres[-1]
genres

In [ ]:
yor=list(vgdata.Year_of_Release.unique())
del yor[-9]
yor

**THE QUESTION**
What are the NA_Sales for a game before it is released in the United States?

We are predicting NA_SALES using Platform, Y_O_R(Maybe), Genre, Publisher(Maybe), Developer, Rating

In [ ]:
vgdata.shape

In [ ]:
print("There are {} rows and {} columns in the dataset".format(vgdata.shape[0],vgdata.shape[1]))

In [ ]:
vgdata.head()

In [ ]:
vgdata.dtypes

continuous
------------------------
NA_SALES(float) 

categorical
------------------------
Platform(string) 
Y_O_R(float)(Maybe) 
Genre(string) 
Publisher(string)(Maybe)
Developer(string)
Rating(string)

In [ ]:
vgdata.isnull().sum()

In [ ]:
vgdata.describe()

In [ ]:
print('Missing Values %')
print("-"*25)
print(round(vgdata.isnull().sum()/vgdata.shape[0]*100,2))

In [ ]:
#Sorting
vgdata.sort_values('NA_Sales',ascending=False).head(5)

In [ ]:
data=vgdata.Rating.value_counts()
data.plot(kind='pie')

E rating has the most games, followed by T.

In [ ]:
data=vgdata.Developer.value_counts()[0:20]
data.plot(kind='bar',figsize=(20,5))

In [ ]:
data=vgdata.Year_of_Release
data.plot.hist(by='Year_of_Release',bins=39,figsize=(20,5))

In [ ]:
data=vgdata.Platform.value_counts()[0:20]
data.plot(kind='bar',figsize=(20,5))

In [ ]:
data=vgdata.Genre.value_counts()[0:20]
data.plot(kind='bar',figsize=(20,5))

In [ ]:
data=vgdata.Publisher.value_counts()[0:20]
data.plot(kind='bar',figsize=(20,5))

In [ ]:
vgdata['NA_Sales'].plot(kind='kde',figsize=(20,5))
plt.xlim({1,-1})

In [ ]:
nicedata=vgdata[vgdata['NA_Sales']<0.5]
nicedata=nicedata[nicedata['NA_Sales']>-0.5]
fig=plt.gcf()
sns.boxplot(x='Rating',y='NA_Sales',data=nicedata)
fig.set_size_inches(20, 5)

In [ ]:
sns.lmplot(x='Year_of_Release',y='NA_Sales',data=vgdata,size=10)

In [ ]:
#CLEAN DATASET
#Remove or replace NULL VALUES
#One-Hot Encode/Label Encode Categorical columns

In [ ]:
vgdata.drop(["Critic_Score","Critic_Count","User_Score","User_Count","Global_Sales",'JP_Sales',"EU_Sales","Other_Sales"],
            axis=1,inplace=True)

In [ ]:
vgdata.head()

In [ ]:
vgdata.dropna(subset=['Name'],inplace=True)
vgdata.dropna(subset=['Year_of_Release'],inplace=True)
vgdata.dropna(subset=['Publisher'],inplace=True)
vgdata['Developer'].fillna('None',inplace=True)
vgdata['Rating'].fillna('None',inplace=True)

In [ ]:
vgdata.isnull().any()

In [ ]:
columns=['Platform','Year_of_Release','Genre','Publisher','Developer','Rating']
for i in columns:
    print(i)
    print("_"*20)
    print(vgdata[i].nunique())
    print('\n')
    

In [ ]:
#OHE Genre & Rating
#LABEL ENCODE everything else

vgdata=pd.concat([vgdata,pd.get_dummies(vgdata.Rating)],axis=1)
vgdata=pd.concat([vgdata,pd.get_dummies(vgdata.Genre)],axis=1)
vgdata.drop(['Genre','Rating'],axis=1,inplace=True)

In [ ]:
Dev_LE=LabelEncoder()
vgdata.Developer=Dev_LE.fit_transform(vgdata.Developer)

Pub_LE=LabelEncoder()
vgdata.Publisher=Pub_LE.fit_transform(vgdata.Publisher)

YOR_LE=LabelEncoder()
vgdata.Year_of_Release=YOR_LE.fit_transform(vgdata.Year_of_Release)

Plat_LE=LabelEncoder()
vgdata.iloc[:,1]=Plat_LE.fit_transform(vgdata.iloc[:,1])

vgdata.rename(columns={vgdata.columns[1]:'System'}, inplace=True)
vgdata.head()

In [ ]:
Y=vgdata['NA_Sales']
X=vgdata
X.drop(['NA_Sales'],axis=1,inplace=True)

In [ ]:
X.head()
X.drop(['Name'],axis=1,inplace=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2,random_state=777)

In [ ]:
LR=LinearRegression()
#fit
LR.fit(X_train,y_train)
#predict
pred = LR.predict(X_test)
print(mean_squared_error(pred,y_test))